# The paleopiezometry module

📣 The GrainSizeTools script includes a **Jupyter notebook template** to promote the reproducibility of palaeopizometry studies. Use it as a template by deleting and adding as necessary and use it as supplementary material to your study so that anyone can reproduce your results.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Estimate the stress using the average grain size

### Before you begin

The paleopiezometry module includes a function named ``calc_diffstress()`` for estimating differential stress based on "average" recrystallized grain sizes. The function requires careful consideration of the following factors for correct differential stress estimation:

1. definition of the mineral phase and the piezometer relationship to be used
2. enter the (apparent) grain size as **equivalent circular diameters** in microns
3. enter a specific average (arithmetic mean, median, etc.) with **no stereological correction**,
4. set the stress framework, either uniaxial compression/extension or plane stress, for proper stress correction.

Regarding the first factor, GrainSizeTools includes a database of paleopiezometric relations, including the most common mineral phases quartz, calcite, olivine and albite feldspar. In addition, the script facilitates the writing of ad hoc piezometric relations. As we will see later, it is also very easy to add new piezometric relationships.

For the second factor, the function assumes by default that the average grain size you specify has been estimated using equivalent circular diameters (ECDs). As some piezometric relationships, especially those established longer ago, were established using linear intercepts (LI) rather than ECDs, the function will automatically convert ECDs to linear intercepts if necessary using the Hoff and Rhines (1968) correction. This means that **you do not need to worry about whether the piezometer was originally calibrated using linear intercepts**, always use averages based on equivalent circular diameters in microns. The function will explicitly warn you when using this ECD ti LI of conversion.

The third factor is key for the correct estimation of differential stress, as each paleopiezometry relationship has been calibrated to a specific average grain size (e.g. the arithmetic mean, median or RMS mean) and thus **only gives valid results if the same type of average is used**. In addition, **you must not apply any type of stereological correction to the calculated grain average**, if the author(s) of the piezometer used any type of stereological correction during calibration, your input will be automatically corrected by the function.

The fourth factor means that the user must decide whether or not to correct the differential stress estimate for plane strain using the correction factor suggested by Paterson and Olgaard (2000). The reason for this is that piezometer calibration experiments are mainly performed in uniaxial compression, whereas natural shear zones behave approximately like plane-strain volumes.

### Use of the piezometric database

In [1]:
# Load the script first (change the path to GrainSizeTools_script.py accordingly!)
%run C:/Users/marco/Documents/GitHub/GrainSizeTools/grain_size_tools/GrainSizeTools_script.py

module plot imported
module averages imported
module stereology imported
module piezometers imported
module template imported

Welcome to GrainSizeTools script
A free open-source cross-platform script to visualize and characterize grain size
population and estimate differential stress via paleopizometers.

Version: 2023.11.xx
Documentation: https://marcoalopez.github.io/GrainSizeTools/

Type get.functions_list() to get a list of the main methods



You can get information from the console on the different available piezometric relations  just by typing ``piezometers._()``, where \_ is the mineral phase, either ``quartz``, ``calcite``, ``olivine``, or ``feldspar``. For example:

In [2]:
piezometers.quartz()

Available piezometers:
'Cross'
'Cross_hr'
'Holyoke'
'Holyoke_BLG'
'Shimizu'
'Stipp_Tullis'
'Stipp_Tullis_BLG'
'Twiss'


If you want to get the details of a particular piezometric relationship simply pass the name of the relationship inside the parenthesis. Remember that the relationship between recrystallized grain size and differential stress is

$$\sigma_d = B g^{-m}$$

where $\sigma_d$ and $g$ are the differential stress and the average grain size respectively.

In [3]:
piezometers.quartz('Twiss')

(550,
 0.68,
 'Ensure that you entered the apparent grain size as the arithmetic mean grain size',
 True,
 1.5)

The five different outputs separated by commas correspond with:

- the constant *B* of the piezometric relation
- the exponent *m* of the piezometric relation
- A warning indicating the average to use with this piezometric relation
- An indication of whether the piezometric relation was calibrated using linear intercepts (if ``False`` the piezometric relation was calibrated using equivalent circular diameters).
- The stereological correction factor used (if applicable). If ``False``, no stereological correction applies.

### Estimating differential stresses

Let us first look at the documentation of the:

In [4]:
calc_diffstress?

Signature: calc_diffstress(grain_size, phase, piezometer, correction=False)
Docstring:
Apply different piezometric relations to estimate the differential
stress from average apparent grain sizes. The piezometric relation has
the following general form:

df = B * grain_size**-m

where df is the differential stress in [MPa], B is an experimentally
derived parameter in [MPa micron**m], grain_size is the aparent grain
size in [microns], and m is an experimentally derived exponent.

Parameters
----------
grain_size : positive scalar or array-like
    the apparent grain size in microns

phase : string {'quartz', 'olivine', 'calcite', or 'feldspar'}
    the mineral phase

piezometer : string
    the piezometric relation

correction : bool, default False
    correct the stress values for plane stress (Paterson and Olgaard, 2000)

 References
-----------
Paterson and Olgaard (2000) https://doi.org/10.1016/S0191-8141(00)00042-0
de Hoff and Rhines (1968) Quantitative Microscopy. Mcgraw-Hill. New 

As indicated in the documentation, the ``calc_diffstress()`` requires at least three inputs: (1) the average grain size in microns, (2) the mineral phase, and (3) the piezometric relation to be used. A few examples are given below:

In [5]:
calc_diffstress(12, phase='quartz', piezometer='Twiss')

differential stress = 83.65 MPa

INFO:
Ensure that you entered the apparent grain size as the arithmetic mean grain size
ECD was converted to linear intercepts using de Hoff and Rhines (1968) correction


The function returns the calculated differential stress (in MPa) plus some relevant information about the corrections made and the type of average expected as input.

As most piezometric calibrations have been calibrated using uniaxial compression deformation experiments, let's correct this estimate for planar stress using the correction suggested by Paterson and Olgaard (2000). This is done by passing a new parameter within the function as follows (note the slightly different value of differential stress):

In [6]:
calc_diffstress(12, phase='quartz', piezometer='Twiss', correction=True)

differential stress = 96.59 MPa

INFO:
Ensure that you entered the apparent grain size as the arithmetic mean grain size
ECD was converted to linear intercepts using de Hoff and Rhines (1968) correction


## How to synthesise a palaeopizometric dataset

> ⚠️ **Never combine means -or any other statistic- as if they were data points**. A common mistake is to take different piezometric estimates and averaged them. However, the stress estimates are already an average and calculating a mean of the averages is mathematically incorrect. Similarly, to look at the raw grain size data from all the grain maps and make the estimate from there is also mathematically incorrect. The first approach ignores the uncertainty in any estimate of the average and the second ignores the uncertainty within and between experimental conditions (or measured areas). The correct procedure is to take the weighted mean of the averages, where each mean is weighted by its variance.

In [3]:
import sys
from datetime import date    
today = date.today().isoformat()
import matplotlib as mpl

print(f'Notebook tested in {today} using:')
print('Python', sys.version)
print('Numpy', np.__version__)
print('Matplotlib', mpl.__version__)

Notebook tested in 2023-12-22 using:
Python 3.10.13 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:15:57) [MSC v.1916 64 bit (AMD64)]
Numpy 1.26.2
Matplotlib 3.8.0
